In [816]:
using RemoteREPL
if !isdefined(Base, :task)
	task = @async serve_repl()
end

Task (failed) @0x00000000915e0bb0
IOError: listen: address already in use (EADDRINUSE)
Stacktrace:
 [1] uv_error
   @ .\libuv.jl:97 [inlined]
 [2] #listen#13
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:629 [inlined]
 [3] #listen#10
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:622 [inlined]
 [4] #listen#12
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:626 [inlined]
 [5] listen
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:626 [inlined]
 [6] serve_repl(address::Sockets.IPv4, port::Int64; kws::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
   @ RemoteREPL C:\Users\stebb\.julia\packages\RemoteREPL\tVFcu\src\server.jl:241
 [7] serve_repl (repeats 2 times)
   @ C:\Users\stebb\.julia\packages\RemoteREPL\tVFcu\src\server.jl:240 [inlined]
 [8] (::var"#900#

In [817]:
using Dates, DataFrames
using Plots, PlotThemes, Printf
theme(:dracula)
plotlyjs()
using Unitful, Latexify, Roots
using PhysicalConstants.CODATA2018: g_n

### Read and filter the Data

In [818]:
sitestr = read(download("https://www.landsvirkjun.is/rennsli-um-yfirfall-halslons"), String)
arrReg = r"\[(\[\d+,\d*[.]*\d*\][,\]])+"
nameReg = r"""\u005C"name\u005C":\u005C"(\w*\s\d*)*\u005C".\u005C"data\u005C":"""

datas = [sitestr[i] for i in findall(arrReg, sitestr)]
datas = [
	replace(i[2:end-1], "],[" => ";","," => " ")
	for i in datas
]
datas = eval.(Meta.parse.(datas))
names = [sitestr[i][end-15:end-12] for i in findall(nameReg, sitestr)];

# OH MY GOD THIS IS SO EASY!!!!!!!!
datas[1] = vcat(datas[1], zeros(length(datas[2][:,1]) - length(datas[1][:,1]),2))
datas[1][:,1] = cumsum(vcat(datas[1][1],ones(122)*diff(datas[1][:,1])[1]))
datas = hcat(datas...)
dates = [names[i] .* " Date" for i in eachindex(names)]
volsecs = [names[i] .* " volsecs" for i in eachindex(names)]
namess = Vector{String}(undef, 10)
for i in eachindex(namess)
	namess[i] = i % 2 == 0 ? volsecs[i÷2] : dates[i÷2+1]
end
dates = nothing
volsecs = nothing
datas = DataFrame(datas, namess)
dagar = datas
tvoþ_22 = datas[:,1]
tvoþ_21 = datas[:,2]
tvoþ_20 = datas[:,3]
tvoþ_19 = datas[:,4]
tvoþ_18 = datas[:,5]


123-element Vector{Float64}:
 1.6566336e12
 1.65672e12
 1.6568064e12
 1.6568928e12
 1.6569792e12
 1.6570656e12
 1.657152e12
 1.6572384e12
 1.6573248e12
 1.6574112e12
 1.6574976e12
 1.657584e12
 1.6576704e12
 ⋮
 1.666224e12
 1.6663104e12
 1.6663968e12
 1.6664832e12
 1.6665696e12
 1.666656e12
 1.6667424e12
 1.6668288e12
 1.6669152e12
 1.6670016e12
 1.667088e12
 1.6671744e12

In [819]:
n = length(datas[1,:])
for i in eachindex(datas[1, 1:2:10])
	datas[!,i] = unix2datetime.(datas[!,i]./1000)
end
datas[!,2:2:n] = datas[!,2:2:n] .* 1u"m^3 * s^-1"

,2022 volsecs,2021 volsecs,2020 volsecs,2019 volsecs,2018 volsecs
,Quantity…,Quantity…,Quantity…,Quantity…,Quantity…
1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
2,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
3,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
4,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
5,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
6,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
7,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
8,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1
9,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1,0.0 m^3 s^-1


In [820]:

# I HATE DATES!
trunc(s) = s[end-4:end]
xTicks = (datas[1,1]:Day(14):datas[end,1], trunc.(string.(Date(datas[1,1]):Day(14):Date(datas[end,1]))))
p = plot(Matrix(datas[!,1:2:n]), Matrix(datas[!,2:2:n]),
	minorticks = 14,
	xticks = xTicks,
	labels = reshape(names, 1,5),
	xlabel = "Date",
)


In [821]:
latexify(:(Re = U .* D ./v), env=:eq)
#=
	Re = Reynolds number
	U  = mean velocity
	D  = hydrolic diamiter
	v  = kinematic viscosity of liquid
=#

L"\begin{equation}
Re = \frac{U \cdot D}{v}
\end{equation}
"

In [822]:
latexify(:(f=0.25/log10(k/3.7D+5.74/Re^0.9)^2), env=:eq)
#=
	f = Darcy friction factor
	k = roughness factor
=#

L"\begin{equation}
f = \frac{0.25}{\left( \log_{10}\left( \frac{k}{3.7 \cdot D} + \frac{5.74}{Re^{0.9}} \right) \right)^{2}}
\end{equation}
"

> Swamee-Jain equation for Darcy friction factor
> -- <cite>[Swamee, P.K.; Jain, A.K. (1976). "Explicit equations for pipe-flow problems". Journal of the Hydraulics Division. 102 (5): 657–664][1]</cite>

[1]: https://doi.org/10.1061%2FJYCEAJ.0004542

In [823]:
latexify(:(1/sqrt(f) + 2log10(k/3.7D+2.51/(Re*sqrt(f)))=0), env=:eq)
#=
	f = Darcy friction factor
	k = roughness factor
=#

L"\begin{equation}
\frac{1}{\sqrt{f}} + 2 \cdot \log_{10}\left( \frac{k}{3.7 \cdot D} + \frac{2.51}{Re \cdot \sqrt{f}} \right) = 0
\end{equation}
"

> Original non-linear version : Colebrook–White equation
> -- <cite>[Colebrook, C F (1939). "Turbulent Flow in Pipes, with Particular Reference to the Transition Region Between the Smooth and Rough Pipe Laws". Journal of the Institution of Civil Engineers. 11 (4): 133–156.][1]</cite>

[1]: https://doi.org/10.1680%2Fijoti.1939.13150

In [824]:
latexify(:(h_loss = f * (L/D) * (U^2/2g)), env=:eq)
#=
	h_loss = head loss
	f = Darcy friction factor
	L = pipelength
	D = hydrolic diamiter
	U = mean velocity
	g = gravity acc
=#

L"\begin{equation}
h_{loss} = f \cdot \frac{L}{D} \cdot \frac{U^{2}}{2 \cdot g}
\end{equation}
"

> Darcy Equation 

In [825]:
latexify([:(y=2^0.25/(2sqrt(1+z^2)-z)^(3/8)*(Q*n/sqrt(S_f))^(3/8)), :(b=2y*(sqrt(1+z^2)-z)), :(t=2y*(sqrt(1+z^2)))],env=:eq) |> latexify

#=
	y = height of trapezoid
	Q = Flow rate
	z = side slope
	n = roughness coefficient
	S_f = longitudinal slope
	b = bottom width of trapezoid
	t = top width of trapezoid
=#

L"\begin{equation}
\left[
\begin{array}{c}
y = \frac{2^{0.25}}{\left( 2 \cdot \sqrt{1 + z^{2}} - z \right)^{\frac{3}{8}}} \cdot \left( \frac{Q \cdot n}{\sqrt{S_{f}}} \right)^{\frac{3}{8}} \\
b = 2 \cdot y \cdot \left( \sqrt{1 + z^{2}} - z \right) \\
t = 2 \cdot y \cdot \sqrt{1 + z^{2}} \\
\end{array}
\right]
\end{equation}
"

> ideal trapezoid equation from 
> -- <cite>[MONADJEMI, Parviz. General formulation of best hydraulic channel section. Journal of Irrigation and Drainage Engineering, 1994, vol. 120, no 1, p. 27-35.][2]</cite>

[2]: https://doi.org/10.1061/(ASCE)0733-9437(1994)120:1(27)

In [826]:
latexify(:(n = 0.012*k^(1/6)), env=:eq)

L"\begin{equation}
n = 0.012 \cdot k^{\frac{1}{6}}
\end{equation}
"


> relative surface roughness to manning coefficient conversion
> -- <cite>[Hydraulic roughness – links between Manning’s coefficient, Nikuradse’s equivalent sand roughness and bed grain size][2]</cite>

[2]: https://www.researchgate.net/publication/47529621_Hydraulic_roughness_-_links_between_Manning%27s_coefficient_Nikuradse%27s_equivalent_sand_roughness_and_bed_grain_size

In [827]:
val(x) = (ustrip∘upreferred)(x)
function headloss(
	L = 35_000u"m", # canal length ish
	S = 43u"m", # drop distance
	z = 0.2, # side slope
	k = 0.025u"mm", # Relative Surface Roughness
	)
	S_f = S/L
	m = length(datas[1,:])
	desflow = (maximum∘Matrix)(datas[:,2:2:m])
	n = 0.012*ustrip(k)^(1/6) # manning roughness Coef

	
	# calculate best canal
	y=val(2^0.25/(2sqrt(1+z^2)-z)^(3/8)*(desflow*n/sqrt(S_f))^(3/8))u"m" * 1.5
	b=2y*(sqrt(1+z^2)-z)
	t=2y*(sqrt(1+z^2))
	
	# canal crosssectional area
	A = y/2*(b+t)
	perim = b+2(hypot(t-b,y))
	
	U = datas[:,2:2:m] ./ A # Average velocity in canal
	D = 4A/perim # hydraulic diameter

	v = 1.52e-6u"m^2/s" # kinematic viscosity of water at 4°C
	Re = U .* D ./v # Reynolds number

	# f=0.25 ./ log10.(k ./ (3.7 .* D) .+ 5.74 ./ Re .^ 0.9) .^ 2 # Darcy friction factor
	
	cweqfunc(f, p) = 1/sqrt(f) + 2log10(k/3.7D + 2.51 / (p * sqrt(f)))
	Z = ZeroProblem(cweqfunc, 0.01)
	f = solve.(Z , Re)

	h_loss = f .* L ./ D .* U.^2 ./ 2g_n
	return (h_loss, val.([y,b,t])...)
end

h1,y1,b1,t1 = headloss(35_000u"m", 43u"m", 0.2)
h2,y2,b2,t2 = headloss(7_000u"m", 560u"m", 0.2)
plot(Matrix(h1), title=@sprintf("first y=%.2f m, b=%.2f m, t=%.2f m <br> second y=%.2f m, b=%.2f m, t=%.2f m",y1,b1,t1,y2,b2,t2))
nethead= (Matrix(h1)+Matrix(h2))  #Fá net head gildi 
plot(Matrix(datas[!,1:2:n]),nethead, #Plotta nethead sem fall af dagseteningu
minorticks = 14,
xticks = xTicks,
title="Head loss yfir tímabil",
labels = reshape(names, 1,5),
xlabel = "Date",
)
savefig("afl.png")

In [828]:
using PlotlyJS
η = 0.7533
hours = 8765.81277
flow = datas[!,2:2:10].*1000u"kg/m^3"

P = flow.*g_n.*nethead.*η
P = P.*60 .*60 .*24 .*u"s"
P = uconvert.(u"J",P)
P = Matrix(P)
P[isnan.(P)] .= 0u"s*W"
P = P.*0.0000002778
P = ustrip.(P)
w_ari = [sum(P[:,i]) for i in eachindex(P[1,:])] #leggja saman öll gildin í hverjum dálk til þess að fá summu yfir hvert ár
ar = [2018,2019,2020,2021,2022] #Array sem hýsir öll árin til þess að nota fyrir plot skipunina
bar(ar,reverse(w_ari), #Nota bar plot til þess að fá svona súlurit, síðan eru bara skreytingar
title="Afl fyrir hvert ár",
labels = "Afl",
xlabel = "Date",
ylabel = "kWh")
savefig("aflfyrirar.png")



In [834]:
P = flow.*g_n.*nethead.*η
latexify(:(P = m * g * H_net * η), env=:eq)


L"\begin{equation}
P = m \cdot g \cdot H_{net} \cdot \eta
\end{equation}
"

In [840]:
vetni = []
for i in 1:5
    append!(vetni,(w_ari[i]/48)/1000) #Margfalda öll stök í w_ari með réttum gildum og set inní vetni arrayið
end
vetni
bar(ar,reverse(vetni),ylabel="kg",title="Möguleg vetnis frammleiðsla")
savefig("vetniaari.png")


